# <center>Алгоритм Adaboost</center>

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv('data_adaboost.csv', header=None)

In [3]:
data

,0,1,2
0,1.3,4.0,1
1,1.0,2.0,1
2,2.0,2.5,1
3,2.0,3.5,-1
4,2.5,0.5,1
5,2.3,1.5,1
6,2.3,4.0,-1
7,3.0,2.0,1
8,3.0,3.5,-1
9,4.0,0.5,-1


In [4]:
X = data[[0,1]].values
y = data[2].values

**Задание.** Реализуйте алгоритм Adaboost.

В качестве базового алгоритма возьмем `DecisionTreeClassifier` глубины 1. Количество базовых алгоритмов $T=3$.

- В переменной `self.sample_weight` будем хранить текущие значения весов объектов.
- В методе `fit` последовательно будет производиться построение пней решения. На первом шаге алгоритма веса всех объектов одинаковы, т.е.:
$$w_i^0=\frac{1}{l}, i=1,\ldots,l,$$
где $l$ &ndash; количество объектов обучающей выборки.
- На каждом шаге алгоритма необходимо обучить пень решения с текущими весами объектов. Веса объектов задаются в методе `fit(X, y, sample_weight=...)` пня.
- После того, как пень обучен, вычислить:
$$\epsilon_t=\sum_{i=1}^{l} w_i[y_i \neq b_t(x_i)],$$
$$\alpha_t=\frac{1}{2}\log\frac{1-\epsilon_t}{\epsilon_t}.$$
- И обновить веса объектов:
$$w_i^t=w_i^{t-1}\exp(-\alpha_t y_i b_t(x_i)), i=1,\ldots,l,$$
$$w_i^t=\frac{w_i^t}{\sum_{j=1}^{l} w_j^t}.$$

- Обученные пни будем хранить в переменной `self.trees`.

1. Какими оказались веса объектов после обучения?
2. Пользуясь полученной композицией алгоритмов, классифицируйте точку (4,4) (метод `predict` в реализации).
3. За сколько шагов алгоритм классифицирует все точки верно, если глубина дерева равна 2?

In [5]:
from sklearn.base import BaseEstimator

class AdaboostClassifierCustom(BaseEstimator):
    def __init__(self, n_estimators=3, max_depth=1, random_state=17):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.trees = []
        self.sample_weight = []

    def fit(self, X, y):
        #На первом шаге алгоритма веса всех объектов одинаковы
        self.sample_weight = np.ones(X.shape[0]) / X.shape[0]
        
        for t in range(self.n_estimators):
            #обучение пня. Веса объектов задаются в методе fit(X, y, sample_weight=...) пня
            clf = DecisionTreeClassifier(max_depth=self.max_depth)
            clf.fit(X, y, sample_weight = self.sample_weight )
            
            #вычисляем b,е,а
            b =clf.predict(X)
            e=np.sum(self.sample_weight[b!=y])
            a=np.log((1 - e)/e)/2
            #обновляем веса
            self.sample_weight = self.sample_weight * np.exp(-a*y*b) #обновляем веса
            self.sample_weight = self.sample_weight/np.sum(self.sample_weight) #нормируем веса
            
            #обученные пни будем хранить в переменной self.trees
            self.trees.append(clf) 

        return self
    
    def predict(self, X):
        pred=[]
        for i in range(self.n_estimators):
            pred.append(self.trees[i].predict(X))
        res=np.sign(sum(pred))
        if self.n_estimators% 2 == 0:
            if np.count_nonzero(y == 1)>len(y)-np.count_nonzero(y == 1):
                res[res==0]=1
            else:
                res[res==0]=-1
        return res

In [6]:
model=  AdaboostClassifierCustom()
model.fit(X,y)

AdaboostClassifierCustom()

In [7]:
y_pred = model.predict(X)
print(y)
print(y_pred)

[ 1  1  1 -1  1  1 -1  1 -1 -1 -1 -1]
[ 1  1  1 -1  1  1 -1  1 -1 -1 -1 -1]


In [8]:
accuracy_score(y, y_pred)

1.0

1. Какими оказались веса объектов после обучения?

In [9]:
model.sample_weight

array([0.07291667, 0.02083333, 0.125     , 0.0625    , 0.125     ,
       0.125     , 0.0625    , 0.125     , 0.0625    , 0.07291667,
       0.07291667, 0.07291667])

2. Пользуясь полученной композицией алгоритмов, классифицируйте точку (4,4) (метод `predict` в реализации).

In [10]:
y_pred2 = model.predict([[4,4]])
y_pred2 

array([-1], dtype=int64)

3. За сколько шагов алгоритм классифицирует все точки верно, если глубина дерева равна 2?

In [11]:
acc=0
i=0
while acc!=1:
    i=i+1
    model3=AdaboostClassifierCustom(max_depth=2,n_estimators=i)
    model3.fit(X,y)
    y_pred3 = model3.predict(X)
    acc=accuracy_score(y, y_pred3)
    print('Это шаг',i,' Точность = ',acc)

Это шаг 1  Точность =  0.9166666666666666
Это шаг 2  Точность =  0.9166666666666666
Это шаг 3  Точность =  1.0


In [12]:
print('За',i,'шага алгоритм классифицирует все точки верно')

За 3 шага алгоритм классифицирует все точки верно
